In [31]:
!pip install faiss-cpu
!pip install RapidFuzz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import torch

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

DESCRIPTIONS_CSV = "/kaggle/input/unified/UnifiedDescriptions.csv"
df = pd.read_csv(DESCRIPTIONS_CSV)
print(df.shape)
df.head()

Using device: cuda
(3997, 2)


,image_id,description
0,fp_3fe668b00f4f6efa967866bdda12a977,The garmet is a purple gownedress with an asym...
1,fp_95931f0df1f902a79ee6c9a67725d00c,The garments in the image are a green coat and...
2,fp_fbd8ef2564a18d367d19a7220adf4e62,The woman is wearing a white shirt with a blac...
3,fp_ef8017a524be660d50c6e9476672fe22,The image features a pair of black shorts with...
4,fp_71e9b0adb502858db1770a0307f300a5,The man is wearing a blue blazersuit. The blaz...


In [34]:
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

texts = df["description"].tolist()
batch_size = 64

embeddings = model.encode(
    texts,
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  
)
print( embeddings.shape)

dim = embeddings.shape[1]

index = faiss.IndexFlatIP(dim)

index.add(embeddings)
print( index.ntotal)

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

(3997, 384)
3997


In [35]:
faiss.write_index(index, "unifiedDesc.index")

with open("unifiedimage_ids.pkl", "wb") as f:
    pickle.dump(df["image_id"].tolist(), f)

print("done")

done


In [36]:
#trial
def retrieve(query:str, k=5):
    q_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, k)
    results = []
    for score, idx in zip(D[0], I[0]):
        results.append({
            "image_id": image_ids[idx],
            "description": df.loc[idx, "description"],
            "score": float(score)
        })
    return pd.DataFrame(results)

with open("unifiedimage_ids.pkl","rb") as f:
    image_ids = pickle.load(f)

print(retrieve("a red cocktail dress with lace details", k=5))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                              image_id  \
0  fp_431c33b689e586bbec8636f39dda47eb   
1  fp_91c83bf3e8f18db1eb574844c5d6cad8   
2                            indo_3499   
3  fp_aebf8c3ba1b7c23d3dfdd539f7c548f6   
4                            indo_7376   

                                         description     score  
0  The woman is wearing a red dress that is a she...  0.712846  
1  The woman is wearing a red dress that has a sk...  0.706235  
2  The image features a woman wearing a red dress...  0.690323  
3  The woman is wearing a red dress that has a sy...  0.685643  
4  The image features a woman wearing a red dress...  0.677077  
